# Problem Set 7

In [19]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

## 1. Heart Attacks
#### 1.

In [20]:
heart = pd.read_csv('../Data/heart.csv')
heart.head()
heart = heart.drop(['slp','thall','oldpeak'], axis=1)
heart.dropna()
print(heart.output.describe())
print(heart.dtypes)
print(heart.shape)

count    303.000000
mean       0.544554
std        0.498835
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: output, dtype: float64
age         int64
sex         int64
cp          int64
trtbps      int64
chol        int64
fbs         int64
restecg     int64
thalachh    int64
exng        int64
caa         int64
output      int64
dtype: object
(303, 11)


#### 2.

In [21]:
heart.cp = heart.cp.astype('category')
heart.restecg = heart.restecg.astype('category')
heart.caa = heart.caa.astype('category')
m = smf.logit('output ~ age + sex + cp + trtbps + chol + fbs + restecg + thalachh + exng + caa', data=heart).fit()
print(m.summary())
print(m.get_margeff().summary())
### age, chol, fbs, restecg arent significant

Optimization terminated successfully.
         Current function value: 0.355396
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 output   No. Observations:                  303
Model:                          Logit   Df Residuals:                      286
Method:                           MLE   Df Model:                           16
Date:                Mon, 07 Mar 2022   Pseudo R-squ.:                  0.4843
Time:                        04:54:26   Log-Likelihood:                -107.69
converged:                       True   LL-Null:                       -208.82
Covariance Type:            nonrobust   LLR p-value:                 2.759e-34
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.4035      2.469      0.163      0.870      -4.435       5.242
cp[T.1]          1.5343

- Sex: Being a male decreases the chances of heart attack, this number is significant.
- Cp: Chest pain level 3 has the highest chances of bringing a heart attack, CP is significant
- Age: The higher your age, the higher chances of getting a heart attack. This number is not significant.
- Variables such as caa(number of major vessels) from 1-3, sex, trtbps(resting blood pressure), and exng(exercise enduced angina) are all significant and reduce heart attack probabilities. Some of these makes sense, some don't, like resting bloop pressure, angina, and being a male decreasing the chances.

#### 3.

In [22]:
from sklearn.linear_model import LogisticRegression
X = heart[['age','sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'caa']]
y = heart.output
X1 = pd.get_dummies(X, drop_first=True, columns=['cp','restecg','caa'])
m1 = LogisticRegression(penalty='none', solver='newton-cg').fit(X1,y)
print(m1.coef_) 
print(m1.intercept_)

[[ 0.02366509 -2.13671455 -0.02850619 -0.00606629  0.44373485  0.03574638
  -1.16147274  1.53426412  1.69073314  1.89700821  0.4206284  -0.9518241
  -1.85714506 -3.04848106 -2.37405207  0.46761275]]
[0.40345569]


#### 4.

In [23]:
train_y_prob = m1.predict_proba(X1)
train_y_prob1 = train_y_prob[:,1]
train_y_prob1[:10]

array([0.87141187, 0.93795886, 0.9839053 , 0.95509873, 0.78073825,
       0.5426836 , 0.95063116, 0.91914941, 0.84980607, 0.93412302])

In [24]:
train_y_problab = m1.predict(X1)
train_y_problab[:10]
threshold = 0.5
1.0*(train_y_prob1 > threshold) == train_y_problab

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

#### 5.

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(train_y_problab, y)

0.858085808580858

I think I would be comfortable using this model, however it's always important to consider that there are human lives at stake here, so higher accuracy scores are always more dsireable.

#### 6.

In [26]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, train_y_problab)
cm

array([[113,  25],
       [ 18, 147]])

In [27]:
from sklearn.metrics import accuracy_score
a = accuracy_score(y, train_y_problab)
from sklearn.metrics import precision_score
p = precision_score(y, train_y_problab)
from sklearn.metrics import recall_score
r = recall_score(y, train_y_problab)

print('Accuracy: ',a,'precision: ',p,'recall: ',r)

Accuracy:  0.858085808580858 precision:  0.8546511627906976 recall:  0.8909090909090909


## 2. Predict Airbnb Price
#### 1.

In [69]:
airb = pd.read_csv("../Data/airbnb-beijing-listings.csv.bz2", usecols = ['price','bedrooms','room_type','accommodates','bathrooms'], thousands = ',')
airb['price'] = airb['price'].str.replace(',', '') 
airb['price'] = airb['price'].str.replace('$', '') 
airb['price'] = pd.to_numeric(airb['price'], errors='coerce')
airb.drop(airb.index[airb['price'] == 0], inplace = True) # gets rid of the $0 air b and b's as these are not reasonable
airb = airb.dropna()
airb["bedrooms2"] = pd.cut(airb.bedrooms,                       
                      bins = [0, 1, 2, 3, 4, np.inf],
                      labels = ["0", "1", "2", "3", "4 or more"],
                      right=False) # categorizes the variabel: bedrooms
airb['logarithm'] = np.log(airb['price'])
airb.replace([np.inf, -np.inf], np.nan, inplace=True) # gets rid of the infinite values created by the log function
airb.dropna() # drops the na's created from getting rid of the inifinite values
airb["bathrooms2"] = pd.cut(airb.bathrooms,
                      bins = [0, 1, 2, 3, np.inf],
                      labels = ["0", "1", "2", "3 or more"],
                      right=False)
airb["accommodates2"] = pd.cut(airb.accommodates,
                      bins = [1, 2, 3, 4, np.inf],
                      labels = ["1", "2", "3", "4 or more"],
                      right=False)
m3 = smf.ols("logarithm ~ bedrooms2 + room_type + accommodates2 + bathrooms2", data = airb).fit()
m3.summary()

/tmp/ipykernel_87/1323061788.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  airb['price'] = airb['price'].str.replace('$', '')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              logarithm   R-squared:                       0.462
Model:                            OLS   Adj. R-squared:                  0.462
Method:                 Least Squares   F-statistic:                     2765.
Date:                Mon, 07 Mar 2022   Prob (F-statistic):               0.00
Time:                        05:53:38   Log-Likelihood:                -35568.
No. Observations:               38686   AIC:                         7.116e+04
Df Residuals:                   38673   BIC:                         7.127e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      5.4249      0.047    114.669      0.000       5.332       5.518
bedrooms2[T.1]                 0.0563      0.040      1.413      0.158      -0.022       0.134
bedrooms2[T.2]                 0.1791      0.041      4.389      0.000       0.099       0.259
bedrooms2[T.3]                 0.4472      0.042     10.593      0.000       0.364       0.530
bedrooms2[T.4 or more]         0.7742      0.044     17.602      0.000       0.688       0.860
room_type[T.Private room]     -0.3253      0.007    -44.126      0.000      -0.340      -0.311
room_type[T.Shared room]      -0.9569      0.017    -56.845      0.000      -0.990      -0.924
accommodates2[T.2]             0.3251      0.013     24.221      0.000       0.299       0.351
accommodates2[T.3]             0.3860      0.017     23.288      0.000       0.354       0.418
accommodates2[T.4 or more]     0.5924      0.015     38.401      0.000       0.562       0.623
bathrooms2[T.1]                0.1667      0.024      7.022      0.000       0.120       0.213
bathrooms2[T.2]                0.3387      0.026     13.101      0.000       0.288       0.389
bathrooms2[T.3 or more]        0.8848      0.030     29.666      0.000       0.826       0.943
==============================================================================
Omnibus:                     9155.631   Durbin-Watson:                   1.777
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            55952.994
Skew:                           0.999   Prob(JB):                         0.00
Kurtosis:                       8.543   Cond. No.                         48.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 2.

In [37]:
bnb_pred = m3.predict(airb)
bnb_pred

0        6.803216
1        5.647766
2        5.973020
3        5.973020
4        5.973020
           ...   
38809    5.647766
38810    7.676307
38811    5.915056
38812    5.915056
38813    6.095839
Length: 38686, dtype: float64

#### 3.

In [49]:
bruh = airb['logarithm'].values
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(bnb_pred, bruh))

0.6068148985864634

#### 4.

In [115]:
newX = {"bedrooms2":['2'], 'room_type':['Shared room'], 'accommodates2':['4 or more'], 'bathrooms2':['2']}
two_room = m3.predict(newX)
print('Predicted Price: ',two_room)

Predicted Price:  0    5.578216
dtype: float64


In [116]:
compute = airb[(airb["bedrooms2"] == '2') & (airb['accommodates2'] == '4 or more')]
two_room = m3.predict(compute)
print('Predicted Price: ', np.mean(two_room))      #????

Predicted Price:  6.375890864331857


#### 5.

In [110]:
compute = airb[(airb["bedrooms2"] == '2') & (airb['accommodates2'] == '4 or more')]
print('calculated price: ',compute.logarithm.mean())

calculated price:  6.384548388368513
